<h4>Librerias</h4>

In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h4>Borrado de capas, tablas y relaciones en la SDE</h4>

In [ ]:
ruta_sde = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde"
arcpy.env.workspace = ruta_sde
# Listas
dataframes = arcpy.ListDatasets()
capas = arcpy.ListFeatureClasses()
tablas = arcpy.ListTables()

for dataframe in dataframes:
    arcpy.Delete_management(dataframe)
    print("Se borra dataframe {0}".format(dataframe))
    
for capas in capas:
    arcpy.Delete_management(capas)
    print("Se borra tabla {0}".format(capas))
    
for tabla in tablas:
    arcpy.Delete_management(tabla)
    print("Se borra capa {0}".format(tabla))

In [ ]:
ruta_filegdb = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb"
arcpy.env.workspace = ruta_filegdb

# ? Listado de datasets
lista_datasets = []
for dataset in arcpy.ListDatasets():
    lista_datasets.append(dataset)
    
# ? Almacenamiento de rutas de cada una de las capas que se copiaran a base de datos .sde
# # ** Features into Dataset
listado_rutas_capas_into_dataset = []
for dataset in arcpy.ListDatasets():
    for capas in arcpy.ListFeatureClasses(feature_dataset=dataset):
        ruta_capas_i_ds = os.path.join(ruta_filegdb,dataset,capas)
        listado_rutas_capas_into_dataset.append(ruta_capas_i_ds)

# # ** Features out Dataset
listado_rutas_capas_out_dataset = []
for capas in arcpy.ListFeatureClasses():
    ruta_capas_o_ds = os.path.join(ruta_filegdb,capas)
    listado_rutas_capas_out_dataset.append(ruta_capas_o_ds)
    
# ** Tables out Dataset
listado_rutas_tablas = []
for tablas in arcpy.ListTables():
    ruta_tablas_o_ds = os.path.join(ruta_filegdb,tablas)
    listado_rutas_tablas.append(ruta_tablas_o_ds)

In [ ]:
# ** Toma el sistema de referencia de una capa en específico de la base de datos
ruta_capa_src = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\municipio"

for dataset_a_crear in lista_datasets:
    arcpy.management.CreateFeatureDataset(ruta_sde, dataset_a_crear, spatial_reference = ruta_capa_src)

# ? Copiado de capas y tablas

arcpy.env.workspace = ruta_sde

# ** Copiado de capas dentro de dataset
for dataset in arcpy.ListDatasets():
        for rutas in listado_rutas_capas_into_dataset:
            if dataset.split(".")[1] == rutas.split("\\")[7]:
                ruta_salida = os.path.join(arcpy.env.workspace,dataset,rutas.split("\\")[8])
                arcpy.management.Copy(rutas, ruta_salida)
                print("Se migra la capa {0} dentro del dataset {1}".format(rutas.split("\\")[8],dataset))
                
# ** Copiado de capas dentro de .sde
for rutas in listado_rutas_capas_out_dataset:
    nombre_salida = rutas.split("\\")[7]
    ruta_salida = os.path.join(arcpy.env.workspace,nombre_salida)
    arcpy.management.Copy(rutas, ruta_salida)
    print("Se migra la capa {0}".format(rutas.split("\\")[7]))
    
# ** Copiado de tablas dentro de .sde
for rutas in listado_rutas_tablas:
    arcpy.management.Copy(rutas, rutas.split("\\")[7])
    print("Se migra la tabla {0}".format(rutas.split("\\")[7]))